In [2]:
hej = "noget andet"

print("hej", hej)

hej noget andet


In [3]:
import requests
from bs4 import BeautifulSoup

#header 
header = {"User-Agent": "Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.84 Safari/537.36"}

#request url

url = "https://www.domstol.dk/svendborg/retslister/2023/2/civile-sager-uge-8/"
r = requests.get(url,headers=header)



    

#Loop - Hent første, anden, tredje URL
    #Html parser
        #find_all + linjerne i listen bliver til kolonner
        
 
        

In [34]:
import re
import pandas as pd
pd.options.mode.chained_assignment = None
from datetime import datetime


columns_and_regex = {
    'ignore': "(Opmærksomheden henledes på).*",
    'empty_line': "(^$)",
    'dato_and_lokale': "\d\d-\d\d-\d\d\d\d kl\.\d\d:\d\d - \d\d:\d\d\..*",
    'Rettens j.nr.': 'Rettens j.nr.:.*',
    'involverede': '(Sags\xc3\xb8gers advokat|Sags\xc3\xb8gtes advokat|Klagers advokat|Sags\xc3\xb8gte|Sags\xc3\xb8ger):.*',
    'dommer': '(Dommer):.*',
    'offentlighed': '(Retsm\xc3\xb8det er offentligt)|(Retsm\xc3\xb8det er ikke offentligt).*',
    'Sagen drejer sig om': 'Sagen drejer sig om:.*',
    }
# title

def get_matching_column (raw_line, prev_matching_column=None):
    # Locate the first matching column in columns_and_regex and return the column title
    if prev_matching_column == "empty_line":
        return "title"
    for key in columns_and_regex:
        regex = columns_and_regex[key]
        # Look for a match against the regex
        found_match = re.match(regex, raw_line)
        print("raw_line")
        print(raw_line)
#         print("key: ")
#         print(key)
#         print("regex")
#         print(regex)
#         print("found_match")
#         print(found_match)

        if found_match:
            # The column name is the value of key
            return key
        else:
            # Go to next key
            continue
        
    # No match. Returning false
    return False
        


#HTML parser = Lister med overblik over URL
soup = BeautifulSoup(r.content, "html.parser")
p_tags = soup.find("div", class_='editor-content').find_all('p')

def parse_p_tag (p_tag):
    removeStrong = re.sub('<strong>|</strong>', '', str(p_tag))
    lines = [_ for _ in removeStrong.split('<br/>')]
    to_return = []
    
    prev_matching_column = ""
    for i in range(len(lines)):
        row = lines[i]

        # Strip row
        row = re.sub('<p(.{0,}?)>|</p>', '', row)
        row = str(row).replace('\\xa0', '').strip()
        
        matching_column = get_matching_column(row, prev_matching_column)
        prev_matching_column = matching_column
        print("matching_column")
        print(matching_column)
#        print(matching_column)
        if i == 1:
            # Dato og lokale
            # Split dato og lokale
            dato_and_lokale = row.split(". ")
            # Row bliver til dato
            date_and_time = ''
            start_date_time = ''
            end_date_time = ''
            
            lokale = ''
            if len(dato_and_lokale) == 2:
                lokale = dato_and_lokale[1]
                date_and_time = dato_and_lokale[0].split(" ", 1)
                date_and_time_split = dato_and_lokale[0].split(" - ")
                date_and_time_without_interval = date_and_time_split[0]
                start_date_time = datetime.strptime(date_and_time_split[0], '%d-%m-%Y kl.%H:%M')
                end_date_time = datetime.strptime(date_and_time_split[1], '%H:%M')
                end_date_time = end_date_time.replace(year=start_date_time.year, month = start_date_time.month, day = start_date_time.day)
                row = end_date_time
            to_return.append(lokale)
            to_return.append(start_date_time)

        elif i == 2:
            # Rettens j.nr.
            row = row.replace('Rettens j.nr.: ', '')
        elif i == 6:
            # Sagen drejer sig om
            row = row.replace('Sagen drejer sig om:', '')
        to_return.append(row)
    return to_return

list_of_lists = []
for i in range(len(p_tags) - 1):
    p = parse_p_tag(p_tags[i])
    
    if (len(p) == 9):
        list_of_lists.append(p)

data_frame = pd.DataFrame(list_of_lists)

# adding column name to the respective columns
data_frame.columns =['Title', 'Lokale', 'Starttidspunkt', 'Sluttidspunkt',  'Rettens j.nr.', 'Offentlighed', 'Dommer', 'Involverede', 'Sagen drejer sig om']

data_frame.sort_values(['Starttidspunkt', 'Sluttidspunkt'],
              ascending = [True, True])

raw_line
Opmærksomheden henledes på, at der kan være nedlagt forbud mod offentlig gengivelse af navn, stilling eller bopæl på sigtede (tiltalte) eller andre under sagen nævnte personer eller mod, at de pågældendes identitet på anden måde offentliggøres (navneforbud).
matching_column
ignore
raw_line

raw_line

matching_column
empty_line
matching_column
title
raw_line

raw_line

matching_column
empty_line
matching_column
title
raw_line
20-02-2023 kl.09:00 - 12:00. Retssal 5.
raw_line
20-02-2023 kl.09:00 - 12:00. Retssal 5.
raw_line
20-02-2023 kl.09:00 - 12:00. Retssal 5.
matching_column
dato_and_lokale
raw_line
Rettens j.nr.: BS-52352/2022-SVE
raw_line
Rettens j.nr.: BS-52352/2022-SVE
raw_line
Rettens j.nr.: BS-52352/2022-SVE
raw_line
Rettens j.nr.: BS-52352/2022-SVE
matching_column
Rettens j.nr.
raw_line
Retsmødet er ikke offentligt
raw_line
Retsmødet er ikke offentligt
raw_line
Retsmødet er ikke offentligt
raw_line
Retsmødet er ikke offentligt
raw_line
Retsmødet er ikke offentligt
raw_

,Title,Lokale,Starttidspunkt,Sluttidspunkt,Rettens j.nr.,Offentlighed,Dommer,Involverede,Sagen drejer sig om
0,"Borgerlig sag, Hovedforhandling med sagkyndige...",Retssal 4.,2023-02-20 10:00:00,2023-02-20 12:00:00,BS-3909/2022-SVE,Retsmødet er ikke offentligt,Dommer: Anders Munch-Jensen,Klagers advokat: Bitten Lisette Kjærsgaard,Forvaltningssag
1,"Borgerlig sag, Hovedforhandling med sagkyndige...",Retssal 4.,2023-02-20 13:00:00,2023-02-20 15:00:00,BS-13763/2022-SVE,Retsmødet er ikke offentligt,Dommer: Anders Munch-Jensen,Klagers advokat: Bitten Lisette Kjærsgaard,Forvaltningssag
2,"Borgerlig sag, Hovedforhandling med sagkyndige...",Retssal 4.,2023-02-21 13:00:00,2023-02-21 15:00:00,BS-51502/2022-SVE,Retsmødet er ikke offentligt,Dommer: Anders Munch-Jensen,Klagers advokat: René Schack,Forvaltningssag
3,"Borgerlig sag, Hovedforhandling med sagkyndige...",Retssal 4.,2023-02-21 13:00:00,2023-02-21 15:00:00,BS-32112/2022-SVE,Retsmødet er ikke offentligt,Dommer: Anders Munch-Jensen,Klagers advokat: René Schack,Forvaltningssag
4,"Borgerlig sag, Retsmøde, Isoleret bevisoptagelse",Retssal 4.,2023-02-24 09:00:00,2023-02-24 10:00:00,BS-6264/2023-SVE,Retsmødet er offentligt,Dommer: Ulla Storm,Sagsøgte: Mads Damgaard Poulsen,Retspleje og civilproces
